## Bibliotecas

In [22]:
import cv2
import numpy as np
import pyautogui
import os
import asyncio
import time

from playwright.async_api import async_playwright
from bs4 import BeautifulSoup

## Carregamento da imagem do botão

In [4]:
button = cv2.imread('botao.png', 0)

pasta_raiz = os.getcwd()
print(pasta_raiz)

/home/arthur/F01


## Coletor auxiliar

In [5]:
async def coletor_html(cidade, subtag, url): 

    playwright = await async_playwright().start()
    browser = await playwright.chromium.launch(headless = True)
    page = await browser.new_page()
    await page.goto(url)
    conteudo = await page.content()

    url_aux = url.replace(".", "")
    url_aux = url_aux.replace("/", "")
    url_aux = url_aux.replace(":", "")
    subtag = subtag.lower()
    
    with open(pasta_raiz+"/"+url_aux + ".html", "w") as file:
        file.write(conteudo)
    file.close()
    
    await page.close()
    await browser.close()

## Coletor com visão computacional

In [86]:
def gerar_posicao_botao(button, screenshot):

    res = cv2.matchTemplate(screenshot, button, cv2.TM_SQDIFF)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    
    return min_loc[0] + button.shape[1]//2 , min_loc[1] + button.shape[0]//2

In [85]:
async def coletor_geral(cidade, subtag, url, loop): 
    
    try:
    
        playwright = await async_playwright().start()
        browser = await playwright.chromium.launch(headless = False)
        page = await browser.new_page()
        await page.goto(url)
        
        await page.screenshot(path=pasta_raiz + "/"+ "screen.png")
        screen = cv2.imread('screen.png', 0)
        positions = gerar_posicao_botao(button, screen)
        
        async with page.expect_download() as download_info:
            await page.mouse.click(positions[0], positions[1])
            
        download = await download_info.value
        await download.save_as(pasta_raiz + "/arquivo.csv")
        
        await page.close()
        await browser.close()
        
    except:
        
        loop.create_task(coletor_html(cidade, subtag, url))

## Execução

In [87]:
loop = asyncio.get_event_loop()
loop.create_task(coletor_geral("Novo Oriente de Minas", "liquidacoes", "https://itabirinha.mg.gov.br/transparencia/pagamentos", loop))

<Task pending name='Task-169' coro=<coletor_geral() running at /tmp/ipykernel_190685/320079238.py:1>>

(581, 283)
-1
0
1
2
3
